In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from scipy.stats import t

# Cargar los datos
df = pd.read_csv("Cientifico de datos M53 - Advertising.csv")

# Definir variables independientes y dependiente
X = df[['TV', 'Radio', 'Newspaper']]
y = df['Sales']

# Dividir en conjunto de entrenamiento (70%) y prueba (30%) con semilla 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Agregar constante para el modelo
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Ajustar el modelo de regresión lineal múltiple
modelo = sm.OLS(y_train, X_train_const).fit()

# Evaluar el ajuste del modelo
print(modelo.summary())

# Predicción para valores específicos
nuevos_datos = np.array([[1, 100, 50, 70]])  # Incluye la constante
prediccion = modelo.predict(nuevos_datos)[0]

# Calcular intervalo de confianza del 90%
alpha = 0.10
df_residual = modelo.df_resid
t_value = t.ppf(1 - alpha/2, df_residual)
std_error = np.sqrt(modelo.mse_resid)
conf_interval = (prediccion - t_value * std_error, prediccion + t_value * std_error)

print(f"t-critica: {t_value:.2f}")
print(f"Predicción de ventas: {prediccion:.2f}")
print(f"Intervalo de confianza del 90%: {conf_interval}")

# Validación de supuestos
residuos = modelo.resid

# Sesgo y curtosis
sesgo = residuos.mean()
curtosis = residuos.kurtosis()
print(f"Sesgo: {sesgo:.4f}, Curtosis: {curtosis:.4f}")

# Estadístico de Durbin-Watson
dw_stat = sm.stats.stattools.durbin_watson(residuos)
print(f"Durbin-Watson: {dw_stat:.4f}")

# Prueba de Heterocedasticidad de White
from statsmodels.stats.diagnostic import het_white
white_test = het_white(residuos, X_train_const)
print(f"Prueba de White (p-valor): {white_test[1]:.4f}")


                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.897
Method:                 Least Squares   F-statistic:                     405.2
Date:                Mon, 19 May 2025   Prob (F-statistic):           1.36e-67
Time:                        12:43:50   Log-Likelihood:                -272.35
No. Observations:                 140   AIC:                             552.7
Df Residuals:                     136   BIC:                             564.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6614      0.368     12.650      0.0

## Conclusiones: 
### El Durbin-Watson es 2.0408, lo que indica que no hay una autocorrelación preocupante.
### El sesgo de los residuos es -0.0000, lo que significa que la distribución es prácticamente simétrica.
### La curtosis es 1.9352, lo que indica que los residuos tienen una distribución platicúrtica.
### La prueba de White tiene un p-valor de 0.0332, lo que indica que hay evidencia de heterocedasticidad.